In [2]:
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

import numpy as np
import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


####  1. Content image, C => image to put style on
####  2. Style image, S
####  3. Generated image, G => Content image with style of S

Using pretrained model, going to take activation of hidden layer somewhere in the middle of the network as result. And Using custom Cost function going to tune the network, to create a C image with style of S, image G **Using aG activation of dimension (1, nH, nW, nC).

# 1. Defining Cost for the neural netwrok

1. Content cost, JC
2. Style cost, SC
3. Combining the two costs, J

## 1.1 Content Cost, JC

Content cost takes a hidden layer activation for both C and G images, aC and aG, and measure their difference. So, when we minimize this cost, it will make Genrated image G contents similar to C image.

In [2]:
def contentCost(aC, aG):
    """
    Computes the content cost
    
    Arguments:
    aC -- tensor of dimension (1, nH, nW, nC)
    aG -- tensor of dimension (1, nH, nW, nC)
    
    Returns: 
    JC -- cost for the content image
    """
    
    # Retrieve dimensions from a_G
    m, nH, nW, nC = aG.get_shape().as_list()
    
    # Reshape aC and aG
    aC_unrolled = tf.reshape(aC, [m, nH * nW, nC])
    aG_unrolled = tf.reshape(aG, [m, nH * nW, nC])
    
    
    JC = (1 / (4 * nH * nW * nC)) * tf.reduce_sum(tf.square(tf.subtract(aC_unrolled, aG_unrolled)))
        
    return JC

## 1.2 Style Cost, JS

Gram matrix G of a set of vectors (v1,…,vn) is the matrix of dot products

Gij = np.dot(vi,vj)

large Gij means vi is very similar to vj

In [1]:
def gramMatrix(A):
    """
    Argument:
    A -- matrix of shape (nC, nH * nW)
    
    Returns:
    GM -- Gram matrix of A, of shape (nC, nC)
    """
    
    GM = tf.matmul(A, A, transpose_b=True)
    
    return GM

need to minimize the distance between the Gram matrix of the style image S and that of the generated image G.

In [8]:
def styleCost(aS, aG):
    """
    Arguments:
    aS -- tensor of dimension (1, nH, nW, nC)
    aG -- tensor of dimension (1, nH, nW, nC)
    
    Returns: 
    JS -- scalar style cost
    """
    
    m, nH, nW, nC = aG.get_shape().as_list()
    
    # Reshape the images
    # if m images, need to use a for loop
    aS = tf.transpose(tf.reshape(aS, [nH * nW, nC]))
    aG = tf.transpose(tf.reshape(aG, [nH * nW, nC]))

    # Computing gram_matrices for S and G
    GMS = gramMatrix(aS)
    GMG = gramMatrix(aG)
    
    
    # Computing the loss (≈1 line)
    JS = (1 / ((2 * nC * nH * nW) ** 2)) * tf.reduce_sum(tf.square(tf.subtract(GMS, GMG)))
        
    return JS

Can get better results if use style cost from different layers and combine them using weighted average.

In [13]:
layerWeights = [('conv1_1', 0.2),
                ('conv2_1', 0.2),
                ('conv3_1', 0.2),
                ('conv4_1', 0.2),
                ('conv5_1', 0.2)]

In [ ]:
def weightedStyleCost(model, layerWeights):
    """
    Computes the overall style cost from several chosen layers
    
    Arguments:
    model -- tensorflow model
    layerWeights -- layers to extract style from with correspinding weights to use
    
    Returns: 
    JS -- weighted style cost
    """
    
    # initialize the overall style cost
    JS = 0

    for layer, w in layerWeights:

        # output tensor of the current layer
        out = model[layer]

        # Set aS to be the hidden layer activation by running the session on out
        aS = sess.run(out)

        # Set aG to be the hidden layer activation from the same layer,
        # going to execute later when we set input as G and run the session
        aG = out
        
        # Compute style_cost for the current layer
        JSlayer = styleCost(aS, aG)

        # Add weighted average to overall style cost
        JS += w * JSlayer

    return JS

Minimizing the weighted style cost will cause the image G to follow the style of the image S

## 1.3 Total Cost, J

weighted average of the content cost and the style cost, using alpha and beta

In [15]:
def totalCost(JC, JS, alpha = 10, beta = 40):
    """
    Computes the total cost
    
    Arguments:
    JC -- content cost
    JS -- weighted style cost
    alpha -- hyperparameter weighting the importance of the content cost
    beta -- hyperparameter weighting the importance of the style cost
    
    Returns:
    J -- total cost
    """
    
    return (alpha * JC) + (beta * JS)

# 2. Optimization

1. Create an Interactive session
2. load content image
3. load style image
4. add random noise to the content image, to create initial generated image
5. load pretrained model
6. build the tensorflow graph
   1. Run the content image through the model and compute the content cost
   2. Run the style image through the model and compute the style cost
   3. Compute the total cost
   4. Define the optimizer and the learning rate
7. initialize the tensorflow graph and run it for no. of iterations while updating genrated image G at every step

#### 1. Create an Interactive session

In [17]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()

#### 2. load content image

In [ ]:
# load content image
C = scipy.misc.imread("images/louvre_small.jpg")

# preprocess the image
C = reshape_and_normalize_image(C)

#### 3. load style image

In [ ]:
# load the style image
S = scipy.misc.imread("images/monet.jpg")

# preprocess the image
S = reshape_and_normalize_image(S)

#### 4. add random noise to the content image, to create initial generated image

In [ ]:
# generate noisy version of the content image
G = generate_noise_image(C)
imshow(G[0])

#### 5. load pretrained model

In [ ]:
# load the pretrained model
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")

#### 6. build the tensorflow graph
   1. Run the content image through the model and compute the content cost

In [ ]:
# Assign the content image to be the input of the model
sess.run(model['input'].assign(C))

# Select the output tensor of layer conv4_2
out = model['conv4_2']

# Set aC to be the hidden layer activation
aC = sess.run(out)

# Set aG to be the hidden layer activation from the same layer,
# going to execute later when we set input as G and run the session
aG = out

# Compute the content cost
JC = contentCost(aC, aG)